# Прототипирование тестового задания на тему "Компьютерное Зрение"      
 ## Выполнил: Адаменко Даниил Дмитриевич
## [ссылка на GitHub](https://github.com/Adam14b/EmotionalRecognition-ImageGenerator)

# Создание и обучение модели "МКЭ"

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## работа с данными:
- [ссылка на данные](https://www.kaggle.com/datasets/msambare/fer2013)

In [4]:
# Параметры
img_width, img_height = 48, 48
batch_size = 64

In [ ]:
# Пути к данным
train_data_dir = r"C:\Users\Adam\Test_Task\data\test"
test_data_dir = r"C:\Users\Adam\Test_Task\data\train"

In [ ]:
# Создание генераторов данных
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical'
)

## Создание модели:

In [5]:
# Создание сверточной модели
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Три категории эмоций
])

/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Компиляция модели
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Обучение и сохранение модели

In [ ]:
# обучение модели
history = model.fit(
    train_generator,
    validation_data=test_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=test_generator.samples // batch_size,
    epochs=200
)

In [5]:
# сохранение модели
model.save(r'C:\Users\Adam\Test_Task\data\model.keras')

# Применение модели:

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd

In [ ]:
# Загрузка обученной модели
try:
    model = tf.keras.models.load_model('/Users/daniiladamenko/Downloads/model.keras')
    print("Модель успешно загружена.")
except Exception as e:
    print(f"Ошибка при загрузке модели: {e}")

In [ ]:
# Захват видео
cap = cv2.VideoCapture(1)  # 0 для первой камеры ######################################
if not cap.isOpened():
    print("Не удалось получить доступ к камере.")
    exit()

In [ ]:
emotion_labels = {0: 'Negative', 2: 'Positive', 1: 'Neutral'}  # "лейблы классов"

In [ ]:
# запуск модели
NegativeAmount = 0
PositiveAmount = 0
NeutralAmount = 0
while True:
    ret, frame = cap.read()
    if not ret:
        print("Не удалось получить кадр.")
        break
    
    # Предварительная обработка кадра
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, (48, 48))
        face = face / 255.0
        face = np.expand_dims(face, axis=0)
        face = np.expand_dims(face, axis=-1)
        
        # Предсказание эмоции
        try:
            prediction = model.predict(face, verbose= 0)
            emotion = np.argmax(prediction)
            label = emotion_labels[emotion]
        except Exception as e:
            print(f"Ошибка при предсказании: {e}")
            continue
        #инкрементирование переменных, отвечающих за количество эмоций
        if (label == 'Negative'): NegativeAmount += 1 
        if (label == 'Positive'): PositiveAmount += 1 
        if (label == 'Neutral'): NeutralAmount += 1 
        cv2.putText(frame,f'удовлетворенность :{round(((0.5*NeutralAmount + PositiveAmount)/(0.5*NeutralAmount + PositiveAmount + NegativeAmount) * 100))}%', (10,60), cv2.FONT_HERSHEY_COMPLEX, 0.9,(255, 255, 0), 2)
        color = (0, 255, 0) if label == 'Positive' else (0, 0, 255) if label == 'Negative' else (255, 255, 0)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    # Показ видео
    cv2.imshow('Video', frame)
    # остановка в случае нажатия на "Q"
    if cv2.waitKey(1) & 0xFF in [ord('q') ]: #можно добавить или изменить клавиши закрытия программы
        break
#вывод таблицы итогов
output = pd.DataFrame.from_dict({'Общая удовлетворенность':[f'{round(((0.5*NeutralAmount + PositiveAmount)/(0.5*NeutralAmount + PositiveAmount + NegativeAmount) * 100))} %'],
    'Негативные эмоции' :[f'{((100 - ((NeutralAmount + PositiveAmount)/(NeutralAmount + PositiveAmount + NegativeAmount) * 100)))} %'],
    'Нейтральне эмоции': [f'{round(((NeutralAmount)/(NeutralAmount + PositiveAmount + NegativeAmount) * 100),4)} %'],
    'Положительные эмоции': [f'{round((PositiveAmount/(NeutralAmount+PositiveAmount+NegativeAmount)*100),4)} %']
    }).T
output.columns = ['Результаты звонка:']
# вывод, очистка памяти, выход
display(output)
cap.release()
cv2.destroyAllWindows()